##### 1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
# p = 1/(1 + e**(-W*X))

In [4]:
def calc_std_feat(x):
  res = (x - x.mean()) / x.std()
  return res

In [5]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [6]:
def calc_logloss(y, y_pred):
    # Переменные, которые отвечают за замену значений сверху и снизу
    instead_of_zero_low = 0.00000001
    instead_of_zero_high = 1 - instead_of_zero_low
    for y_item, y_pred_item in zip(y, y_pred):
        if y_pred_item == 0:
            y_pred_item = instead_of_zero_low
        elif y_pred_item == 1:
            y_pred_item = instead_of_zero_high
        
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [7]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    err = 0
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
#         if i % (iterations / 10) == 0:
#             print(i, W, err)
    return W, err

In [8]:
### Logistic Regression

In [10]:
W, err = eval_model(X, y, iterations=10000, alpha=1e-4)

D:\anaconda3\envs\machine_learning\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
D:\anaconda3\envs\machine_learning\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in multiply
  # This is added back by InteractiveShellApp.init_path()


##### 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [17]:
min_err = 100
best_params = [0, 0]
alfa = 1e-2
W = []
while alfa > 1e-8:
    for iterate in range(1000, 10000, 1000):
        W, err = eval_model(X, y, iterate, alfa)
        if err < min_err:
            min_err = err
            best_params[0] = iterate
            best_params[1] = alfa
    alfa = alfa / 2
print(round(min_err, 7), best_params)

D:\anaconda3\envs\machine_learning\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
D:\anaconda3\envs\machine_learning\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in multiply
  # This is added back by InteractiveShellApp.init_path()
D:\anaconda3\envs\machine_learning\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


0.6042085 [9000, 4.8828125e-06]


##### 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [13]:
def calc_pred_proba(w, x):
    return 1 / (1 + np.exp(-np.dot(W,X)))

In [14]:
W = W.reshape(1, -1)
y_pred_proba = calc_pred_proba(W, X)
y_pred_proba

array([[0.6597323 , 0.54380259, 0.80799447, 0.60321547, 0.40010675,
        0.51797944, 0.67191111, 0.23204442, 0.68647952, 0.69613921]])

##### 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [15]:
def calc_pred(w_l, x_l):
    y_pred_proba = calc_pred_proba(W, X)
    y_pred = []
    for item in y_pred_proba[0]:
        if item >= 0.5:
            y_pred.append(1)
        else:
            y_pred.append(0)
    return y_pred

In [16]:
y_pred = calc_pred(W, X)
y_pred

[1, 1, 1, 1, 0, 1, 1, 0, 1, 1]

##### 5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [32]:
TP = 0
FP = 0
FN = 0
TN = 0

for y_1, y_p in zip(y, y_pred):
    if y_1 == 1 and y_p == 1:
        TP += 1
    elif y_1 == 0 and y_p == 0:
        TN += 1
    elif y_1 == 0 and y_p == 1:
        FP += 1
    elif y_1 == 1 and y_p == 0:  
        FN += 1

print(f'TP = {TP}, FP = {FP}, FN = {FN}, TN = {TN}')

TP = 4, FP = 4, FN = 1, TN = 1


In [33]:
conf_mat = np.zeros([2, 2])
conf_mat

array([[0., 0.],
       [0., 0.]])

In [34]:
conf_mat[0][0] = TP
conf_mat[1][1] = TN
conf_mat[0][1] = FP
conf_mat[1][0] = FN

In [35]:
conf_mat

array([[4., 4.],
       [1., 1.]])

In [42]:
accuracy = (TP + TN) / len(y)
accuracy

0.5

In [45]:
precision = TP / (TP + FP)
precision

0.5

In [46]:
recall = TP / (TP + FN)
recall

0.8

In [47]:
F1 = 2 * (precision*recall) / (precision + recall)
F1

0.6153846153846154

##### 6. Могла ли модель переобучиться? Почему?

Могла, если скорость обучения была выбрана очень большой.

##### 7*. Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [50]:
def eval_model_l1(X, y, iterations, alpha=1e-4, lambda_=1e-8):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    err = 0
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * 2 * np.dot((y_pred - y), X.T) + lambda_ * np.sign(W)) 
#         if i % (iterations / 10) == 0:
#             print(i, W, err)
    return W, err

In [51]:
def eval_model_l2(X, y, iterations, alpha=1e-4, lambda_=1e-8):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    err = 0
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * 2 * np.dot((y_pred - y), X.T) + lambda_ * W) 
#         if i % (iterations / 10) == 0:
#             print(i, W, err)
    return W, err